# Identifying and acquiring relevant X-ray observations

This tutorial will explain the basic concepts and components of the X-ray astronomy Python module 'Democratising Archival X-ray Astronomy' (DAXA). We will particularly focus on the various 'mission' classes (implemented for each of the X-ray telescopes that DAXA supports), and the functionality that allows for large numbers of observations to be selected and downloaded.

DAXA mission classes allow the user to interact with and search various X-ray telescope archives, all through an identical interface, within a single module, and through Python commands. It should be simple for anyone with a passing familiarity with Python to identify and acquire X-ray data relevant to their research.

## Import Statements

In [1]:
from datetime import date

import numpy as np
from astropy.units import Quantity

from daxa.mission import MISS_INDEX, XMMPointed, Chandra, eRASS1DE, eROSITACalPV, NuSTARPointed, Swift, \
    ROSATAllSky, ROSATPointed, ASCA, INTEGRALPointed, Suzaku, XRISMPointed

## What missions are available?

We have implemented support for access and searching the archives of many X-ray telescopes; we would also be willing to provide an interface to the data archives of other X-ray telescopes, if feasible - please feel free to reach out using the support page if you think there is one we should add.

Some telescopes (such as the ROSATPointed/ROSATAllSky and eROSITACalPV/eRASS1DE classes) are not uniquely represented by a single DAXA mission - this is generally the case when the telescope in question has been used in distinctly different ways (e.g. ROSAT had a survey phase and a pointed phase), such that the data for one mode may not be relevant to all applications.

In [2]:
MISS_INDEX

{'xmm_pointed': daxa.mission.xmm.XMMPointed,
 'nustar_pointed': daxa.mission.nustar.NuSTARPointed,
 'erosita_calpv': daxa.mission.erosita.eROSITACalPV,
 'erosita_all_sky_de_dr1': daxa.mission.erosita.eRASS1DE,
 'chandra': daxa.mission.chandra.Chandra,
 'rosat_all_sky': daxa.mission.rosat.ROSATAllSky,
 'rosat_pointed': daxa.mission.rosat.ROSATPointed,
 'swift': daxa.mission.swift.Swift,
 'suzaku': daxa.mission.suzaku.Suzaku,
 'asca': daxa.mission.asca.ASCA,
 'integral_pointed': daxa.mission.integral.INTEGRALPointed,
 'xrism_pointed': daxa.mission.xrism.XRISMPointed}

### XMM-Pointed

This class is for acquiring XMM-Newton data, particularly that taken when the telescope is pointing at specific targets; i.e. it cannot be used to filter and download data taken when the telescope is slewing from one target to the next. XMM is still in use, so the number of observations that are available are constantly increasing - this also means that some data are still in a proprietary period, and DAXA will not be able to access them. **Note that only raw, unprocessed, data can currently be downloaded for XMM using DAXA.**

The three EPIC instruments are supported (PN, MOS1, and MOS2), as well as the two grating spectrometers (RGS1 and RGS2). We also support the acquisition of XMM optical monitor (OM) data, but support for processing it is more limited.


Values that can be passed to the `insts` argument of XMMPointed on declaration (either as single strings or as part of a list):

* PN
* MOS1
* MOS2
* RGS1
* RGS2
* OM

**IMPORTANT NOTE** - We have found that the configuration of some HPCs prevents the AstroQuery module (used as the default way to download XMM data) from connecting to the internet. The XMMPointed mission can also acquire observation information, and data, from the HEASARC archive. If AstroQuery fails with a connection problem, the mission will automatically failover to the HEASARC method, and the user may also manually set the data-acquisition method with the `use_heasarc` argument when setting up the mission.

In [2]:
xm = XMMPointed()

/Users/dt237/code/DAXA/daxa/mission/xmm.py:106: UserWarning: 150 of the 18210 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()


### Chandra

The class for acquiring Chandra data, specifically taken when pointing at a target - Chandra does not have easily accessible 'slew' data, and this class does not consider it at all.

Data from all instruments can be downloaded by DAXA - though unlike XMM there are not data being taken simultaneously, so each ObsID is generally only associated with one instrument. Note though that the gratings (HETG and LETG) are used _with_ another instrument, so cannot be passed by themselves.

**Note that the standard format of Chandra data can be downloaded using DAXA, allowing for the use of the standard Chandra re-processing scripts. This standard download includes pre-made images.**

Values that can be passed to the `insts` argument of Chandra (either as single string or as part of a list):

* ACIS-I
* ACIS-S
* HRC-I
* HRC-S
* LETG [an instrument that this grating was used with must be specified]
* HETG [an instrument that this grating was used with must be specified]

In [4]:
ch = Chandra()

### eROSITA All-Sky DR1 (German Half)

The class for acquiring data from the first data release by the German part of the eROSITA consortium - this covers half the sky to 1/8th the planned final survey depth of eROSITA. All of this data is taken in survey mode, where the telescope is constantly slewing. **Note that the data downloads can include pre-generated images and exposure maps.**

The eROSITA telescope is made up of 7 telescope modules that observe simultaneously, and which can be individually selected on declaration with the `insts` argument - ***it is not recommended to use DAXA to limit the telescope modules being considered, as we crudely modify the event lists to remove events from non-selected telescope modules***.

Values that can be passed to the `insts` argument of eRASS1DE (either as a single string or as part of a list):

* TM1
* TM2
* TM3
* TM4
* TM5
* TM6
* TM7

In [5]:
ea = eRASS1DE()

### eROSITA Calibration and Performance Verification

This class provides DAXA access to the data from the calibration and performance verification stages of the eROSITA mission, including all pointed observations and survey regions (such as the eROSITA Final Equatorial-Depth Survey; eFEDS). **Note that _only_ event list data can be downloaded for this class.**

The eROSITA telescope is made up of 7 telescope modules that observe simultaneously, and which can be individually selected on declaration with the `insts` argument - ***it is not recommended to use DAXA to limit the telescope modules being considered, as we crudely modify the event lists to remove events from non-selected telescope modules***.

Values that can be passed to the `insts` argument of eROSITACalPV (either as a single string or as part of a list):

* TM1
* TM2
* TM3
* TM4
* TM5
* TM6
* TM7

In [6]:
ecpv = eROSITACalPV()

### NuSTAR-Pointed

This class is for acquiring NuSTAR data, particularly that taken when the telescope is pointing at specific targets; i.e. it cannot be used to filter and download data taken when the telescope is slewing from one target to the next. NuSTAR is still in use, so the number of observations that are available are constantly increasing - this also means that some data are still in a proprietary period, and DAXA will not be able to access them. **Note that data downloads can optionally include pre-generated images**

NuSTAR has two instruments that observe simultaneously, and are essentially identical, Focal Plane Module A & B.

Values that can be passed to the `insts` argument of NuSTARPointed on declaration (either as single strings or as part of a list):

* FPMA
* FPMB

In [7]:
nu = NuSTARPointed()

/Users/dt237/code/DAXA/daxa/mission/nustar.py:108: UserWarning: 884 of the 5668 observations located for NuSTARPointed have been removed due to all instrument on-times being zero.
  self._fetch_obs_info()


### XRISM-Pointed

This class is for acquiring XRISM data, particularly that taken when the telescope is pointing at specific targets; i.e. it cannot be used to filter and download data taken when the telescope is slewing from one target to the next. XRISM is still in use, so the number of observations that are available are constantly increasing - this also means that some data are still in a proprietary period, and DAXA will not be able to access them. **Note that data downloads can optionally include pre-generated images**

XRISM has two instruments that can observe simultaneously, but are fundamentally different in their purpose, design, and capabilities. 

The **RESOLVE** instrument has a very low spatial resolution (6x6 pixels, covering ~3 arcmin field of view), but has an exceptionally high spectral resolution; it is a micro-calorimeter instrument, and is more suited to high-resolution spectroscopy of extended objects than any grating spectrometer, both as you don't have to deal with line-broadening due to the dispersion of extended emission (a problem with RGS for instance) and because it has some spatial resolution.

The **XTEND** instrument is a very wide field ($38\times38$ arcmin) CCD spectro-imaging detector, with a much higher spatial resolution than RESOLVE, but vastly lower spectral resolution. The PSF size is typical of missions like Suzaku and ASCA (a bit larger than XMM, and much larger than Chandra).

Values that can be passed to the `insts` argument of XRISMPointed on declaration (either as single strings or as part of a list):

* RESOLVE
* XTEND

In [8]:
xr = XRISMPointed()

/Users/dt237/code/DAXA/daxa/mission/xrism.py:122: UserWarning: 205 of the 421 observations located for XRISM Pointed have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()


### Swift

This class is for acquiring Swift data. Swift is still in use, so the number of observations that are available are constantly increasing. Also, due to the Swift's primary mission of rapid transient follow-up, and how observations are split up, the table of available observations is unusually large (only INTEGRAL is comparable) - **as such this class may take several minutes to declare, depending on your internet connection**.

Swift has three instruments that generally observe simultaneously; the X-ray Telescope (XRT), the Burst Alert Telescope (BAT) which is has poor spatial resolution but has a large field of view and is sensitive to very high energy photons, and the Ultraviolet and Optical Telescope (UVOT). **Note that data downloads can optionally include pre-generated images, but not for BAT observations.**

Values that can be passed to the `insts` argument of Swift on declaration (either as single strings or as part of a list); the XRT and BAT instruments are selected by default:

* XRT
* BAT
* UVOT

In [9]:
sw = Swift()

/Users/dt237/code/DAXA/daxa/mission/swift.py:122: UserWarning: 651 of the 373850 observations located for Swift have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/Users/dt237/code/DAXA/daxa/mission/swift.py:122: UserWarning: 17 of the 373850 observations located for Swift have been removed due to all chosen instrument (XRT, BAT) exposures being zero.
  self._fetch_obs_info()


### ROSAT All-Sky Survey

This class provides access to data taken by ROSAT during its all-sky survey. Though ROSAT had multiple instruments, this was all taken with the position-sensitive proportional counters (PSPC) - specifically with the 'PSPC-C' instrument (ROSAT had two PSPC instruments). The initial all-sky survey was abandoned after an accidental pass over the sun destroyed PSPC-C, but follow-up observations with PSPC-B were taken towards the end of ROSAT's lifetime to complete the survey (**these are not included in this class**).

**Note: Data acquired through this class will include just event lists by default, but can also include pre-generated images and exposure maps.**

In [10]:
ra = ROSATAllSky()

### ROSAT-Pointed

This class provides access to data taken by ROSAT during the pointed phase of its lifetime (including follow-up observations used to complete the all-sky survey). ROSAT instruments could not observe simultaneously, so each separate observation uses a single instrument. **Note: Data acquired through this class will include just event lists by default, but can also include pre-generated images (PSPC & HRI) and exposure maps (just PSPC).**

Values that can be passed to the `insts` argument of ROSATPointed on declaration (either as single strings or as part of a list):

* PSPCB
* PSPCC
* HRI

In [11]:
rp = ROSATPointed()

### Suzaku

This class provides access to data taken by the Suzaku X-ray telescope during pointed observations (data taken while slewing are not included in the public archive). **Note: Data acquired through this class will include just event lists by default, but can also include pre-generated images.**

We provide access to XIS data, but not XRS (as the cooling system was damaged soon after launch) or HXD (as it was not an imaging instrument).

Values that can be passed to the `insts` argument of Suzaku on declaration (either as single strings or as part of a list):

* XIS0
* XIS1
* XIS2
* XIS3

In [12]:
su = Suzaku()

/Users/dt237/code/DAXA/daxa/mission/suzaku.py:109: UserWarning: 14 of the 3055 observations located for Suzaku have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/Users/dt237/code/DAXA/daxa/mission/suzaku.py:297: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EGS' 'GS' 'EGS' ... 'GCL' 'GCL' 'EGS']' has dtype incompatible with int16, please explicitly cast to a compatible dtype first.
  rel_suzaku.loc[type_recog, 'target_category'] = rel_suzaku.loc[type_recog, 'target_category'].apply(


### ASCA

This class provides access to data taken by the ASCA X-ray telescope during pointed observations (we cannot find anywhere to access the data taken whilst slewing). **Note: Data acquired through this class will include just event lists by default, but can also include pre-generated images, spectra, and lightcurves.**


Values that can be passed to the `insts` argument of ASCA on declaration (either as single strings or as part of a list):

* SIS0
* SIS1
* GIS2
* GIS3

In [13]:
asca = ASCA()

/Users/dt237/code/DAXA/daxa/mission/asca.py:125: UserWarning: 5 of the 3079 observations located for ASCA have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()


### INTEGRAL-Pointed

This class is for acquiring INTEGRAL data. INTEGRAL is still in use, so the number of observations that are available are still increasing (_though operations will see around the end of 2024_). The table of available observations is unusually large (only Swift is comparable) - **as such this class may take several minutes to declare, depending on your internet connection**.

INTEGRAl has a selection of instruments that cover different parts of the X-ray and Gamma-ray energy range - most of them are based on the 'coded mask' technology, and so have very limited spatial resolution. **Note that only raw data/calibration files can be downloaded through DAXA, there are no pre-processed images available**.

Values that can be passed to the `insts` argument of INTEGRALPointed on declaration (either as single strings or as part of a list):

* JEMX1
* JEMX2
* ISGRI
* PICsIT
* SPI

In [14]:
inte = INTEGRALPointed()

/Users/dt237/code/DAXA/daxa/mission/integral.py:110: UserWarning: 241 of the 213970 observations located for INTEGRAL have been removed due to all instrument exposures being zero.
  self._fetch_obs_info()
/Users/dt237/code/DAXA/daxa/mission/integral.py:110: UserWarning: 7986 of the 213970 observations located for INTEGRAL have been removed due to all chosen instrument (JEMX1, JEMX2, ISGRI) exposures being zero.
  self._fetch_obs_info()


## Mission properties

Here we run through the basic properties that each of the DAXA mission classes share. We also show examples, particularly in cases where differences between telescopes result in us assigning different values for particular properties.

### Name

The name assigned to each mission class, so that they can be differentiated both by the user and by DAXA functions. Each mission class has two names, the 'internal DAXA name' (used by DAXA to identify missions) and the 'pretty name', which is typically in a more aethsetically pleasing format.

For instance, we show the 'name' and 'pretty name' of the XMMPointed and eROSITA All-Sky Survey 1 classes:

In [15]:
print(xm.name)
print(xm.pretty_name, '\n')

print(ea.name)
print(ea.pretty_name)

xmm_pointed
XMM-Newton Pointed 

erosita_all_sky_de_dr1
eRASS DE:1


### All Instruments & Chosen Instruments

Most telescopes have multiple instruments, though not all are necessarily selected by default. This can be for a number of reasons, but is generally because they either aren't suited to archival/serindipitious science (which is the primary reason this module exists) or because they aren't X-ray telescopes (like the optical monitors on XMM and Swift). 

The instruments whose data is to be acquired are generally specified when the mission class is declared (using the `insts` argument), but can also be set through the `chosen_instruments` property.

Every available instrument for a mission is stored in the `all_mission_instruments` property:

In [16]:
print(rp.all_mission_instruments)
print(ch.all_mission_instruments)

['PSPCB', 'PSPCC', 'HRI']
['ACIS', 'HRC', 'HETG', 'LETG']


The selected instruments (normally specified on declaration) are stored in the `chosen_instruments` property, which can also be set:

In [17]:
print(rp.chosen_instruments)
print(ch.chosen_instruments)
ch.chosen_instruments = ['ACIS-I', 'ACIS-S']
print(ch.chosen_instruments)

['PSPCB', 'PSPCC', 'HRI']
['ACIS', 'HRC']
['ACIS']


/Users/dt237/code/DAXA/daxa/mission/chandra.py:182: UserWarning: Some instrument names were converted to alternative forms expected by this module, the instrument names are now; ACIS
  new_insts = super().check_inst_names(new_insts)


### ObsID Regular Expression

Each of the mission's observations are uniquely identified by an 'ObsID', and each telescope/mission has a different format of ObsID (generally just made up of numeric characters) - there are points where the mission class may have to check the format of a supplied ObsID, and it does that by comparing to the ObsID regular expression:

In [18]:
print(xm.id_regex)
print(rp.id_regex)

^[0-9]{10}$
^(RH|rh|RP|rp|RF|rf|WH|wh|WP|wp|WF|wf)\d{6}([A-Z]\d{2}|)$


### Field of View

The field of view (FoV) values attached to DAXA mission classes represent the half-width (or radius) of the region of sky that a telescope/instrument observes. Given that each telescope instrument tends to have a unique geometry, this is a simplification, but it is beneficial to have a single number that represents how much of the sky an instrument can see.

In the simplest cases, the FoV property is just a single quantity, meaning that there is either only one instrument, or that every instrument has the same field of view. In other cases there may be multiple instruments associated with a mission, in which case they will all have their own entry in a FoV dictionary.

Finally, some telescopes (such as Chandra) have instruments which have irregular geometries (the ACIS-S and HRC-S chips), or that are frequently used in different observational modes that enable and disable different parts of the sensor. As such the actual coverage of the FoV can vary dramatically, in cases like these we will have gone with the half-width of the longest possible side of the field of view.

In [19]:
print(xm.pretty_name, '-', xm.fov, '\n')
print(rp.pretty_name, '-', rp.fov, '\n')
print(sw.pretty_name, '-', sw.fov, '\n')
print(ch.pretty_name, '-', ch.fov)

XMM-Newton Pointed - 15.0 arcmin 

ROSAT Pointed - {'PSPCB': <Quantity 60. arcmin>, 'PSPCC': <Quantity 60. arcmin>, 'HRI': <Quantity 19. arcmin>} 

Swift - {'XRT': <Quantity 11.8 arcmin>, 'BAT': <Quantity 50. deg>, 'UVOT': <Quantity 8.5 arcmin>} 

Chandra - {'ACIS': <Quantity 27.8 arcmin>, 'HRC': <Quantity 49.5 arcmin>}


/var/folders/td/gw9qkx6d3szb1nkt_cfvcbzm000vzl/T/ipykernel_56140/579883568.py:4: UserWarning: Chandra FoV are difficult to define, as they can be strongly dependant on observation mode; as such take these as very approximate.
  print(ch.pretty_name, '-', ch.fov)


### Coordinate Frame

This property contains the coordinate frame for the central positions of observations taken by the mission - this is largely irrelevant to the user, and will be used in cases where the mission class needs to compare an input coordinate to an observation coordinate. 

Also, practically speaking the difference between the ICRS and FK5 frames (most commonly used) is neglible compared to the typical spatial uncertainty involved in X-ray data:

In [20]:
print(ch.coord_frame)
print(asca.coord_frame)

<class 'astropy.coordinates.builtin_frames.icrs.ICRS'>
<class 'astropy.coordinates.builtin_frames.fk5.FK5'>


### Pre-processed Energy Bands

This property will contain the upper and lower energy bounds available for pre-processed data products for a particular mission (if their online dataset supplies energy-bound data products) - these energy bounds are provided on an instrument level (as some missions provide different energy-bound products for different instruments). The left hand column indicates the lower energy bound, and the right hand column the upper energy bound. 

An energy bound being present here does not guarantee that all products supplied by the mission online dataset are available in that bound - e.g. some missions provide bound images and a single, general, exposure map.

In [21]:
rp.preprocessed_energy_bands

{'PSPCB': <Quantity [[0.07, 2.4 ],
            [0.07, 0.4 ],
            [0.4 , 2.4 ]] keV>,
 'PSPCC': <Quantity [[0.07, 2.4 ],
            [0.07, 0.4 ],
            [0.4 , 2.4 ]] keV>,
 'HRI': <Quantity [[0.07, 2.4 ]] keV>}

If the mission in question cannot provide pre-processed data products that are energy bound, then an error will be raised:

In [22]:
inte.preprocessed_energy_bands

PreProcessedNotSupportedError: This mission's archive does not supply pre-processed products within specific energy bands. 

### Observation Information

One of the most important properties of a DAXA mission class - this returns a dataframe of all the observations that are associated with a mission. This can include observations that are not yet publicly available (i.e. they are still in a proprietary period), but will never include observations that are planned but haven't been taken yet. 

In most cases this data is dynamically updated when a mission is declared (i.e. the table is pulled down from a mission server) - this is not the case for eROSITACalPV and eRASS1DE. **Please also note that the Swift and INTEGRALPointed missions have very large `all_obs_info` tables due to the way their data/observations are organised.**

Some information will be constant across telescopes, and some will be mission specific. We present truncated versions of `all_obs_info` for every DAXA mission as of the current date:

In [23]:
date.today().strftime('%d-%B-%Y')

'03-February-2025'

In [24]:
xm.all_obs_info

,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
0,64.925415,55.999440,0000110101,2001-08-19 07:05:23,True,0 days 09:08:33,2002-09-29,310,True,2001-08-19 16:13:56
1,263.674950,-32.581670,0001730201,2001-03-09 12:44:21,True,0 days 04:44:43,2002-05-25,229,True,2001-03-09 17:29:04
2,263.674950,-32.581670,0001730301,2001-03-09 17:30:16,True,0 days 02:36:02,2002-05-25,229,True,2001-03-09 20:06:18
3,263.674950,-32.581670,0001730401,2001-03-09 09:41:25,True,0 days 03:00:59,2002-05-25,229,True,2001-03-09 12:42:24
4,99.349995,6.135278,0001730501,2002-09-17 18:35:28,True,0 days 06:05:39,2004-12-31,508,True,2002-09-18 00:41:07
...,...,...,...,...,...,...,...,...,...,...
17981,261.621875,71.093972,0940870201,2025-01-23 06:07:23,True,0 days 07:35:33,NaT,4601,False,2025-01-23 13:42:56
17982,38.445500,-1.024583,0954191001,2025-01-22 17:42:20,True,0 days 10:50:00,2025-07-30,4601,False,2025-01-23 04:32:20
17983,230.333708,14.069583,0942560501,2025-01-12 00:50:19,True,0 days 04:46:40,NaT,4596,False,2025-01-12 05:36:59
17984,48.432667,-18.110111,0943770501,2025-01-23 23:59:23,True,1 days 03:48:20,NaT,4602,False,2025-01-25 03:47:43


In [25]:
ch.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,instrument,grating,data_mode
0,274.43140,-33.01883,6616,True,True,2006-02-24 04:33:41.000003,2007-09-21 00:26:11.000003,573 days 19:52:30,2007-02-28,GS,ACIS,HETG,CC_000A8
1,83.63292,22.01447,7587,True,True,2007-02-03 09:58:57.000000,2008-07-21 21:53:57.000000,534 days 11:55:00,2008-02-06,SNR,ACIS,HETG,TE_0077C
2,202.50000,47.20000,13814,True,True,2012-09-20 07:21:41.999999,2012-09-22 12:47:41.999999,2 days 05:26:00,2013-10-11,NGS,ACIS,NONE,TE_00958
3,266.41667,-29.00781,13842,True,True,2012-07-21 11:52:41.000002,2012-07-23 17:08:41.000002,2 days 05:16:00,2012-07-25,NGS,ACIS,HETG,TE_008D0
4,316.72458,38.74942,13651,True,True,2012-02-13 20:18:26.999997,2012-02-16 00:51:26.999997,2 days 04:33:00,2013-02-21,MISC,HRC,LETG,DEFAULT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24781,332.17000,45.74231,24644,True,True,2020-09-29 01:23:47.000003,2020-09-29 01:23:47.000003,0 days 00:00:00,2020-09-30,CAL,HRC,NONE,DEFAULT
24782,332.17010,45.74230,1336,True,True,1999-10-03 21:48:53.000004,1999-10-03 21:48:53.000004,0 days 00:00:00,1999-12-14,CAL,HRC,NONE,
24783,54.19708,0.58997,1153,True,True,1999-09-05 04:29:00.999998,1999-09-05 04:29:00.999998,0 days 00:00:00,2002-10-04,CAL,HRC,NONE,SCENTER
24784,84.91458,-69.74361,62437,True,True,1999-09-05 00:50:10.999997,1999-09-05 00:50:10.999997,0 days 00:00:00,2002-03-27,GS,HRC,NONE,SCENTER


In [26]:
ea.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,ra_min,ra_max,dec_min,dec_max,neigh_obs
0,116.703297,42.008289,117048,True,2020-04-15 10:13:39,2020-04-19 02:13:48,3 days 16:00:09,114.725275,118.681319,40.5,43.5,"114045,118045,122045,113048,121048,112051,1160..."
1,120.659341,42.008289,121048,True,2020-04-17 10:14:32,2020-04-21 10:13:37,3 days 23:59:05,118.681319,122.637363,40.5,43.5,"118045,122045,117048,125048,116051,119051,1230..."
2,124.615385,42.008289,125048,True,2020-04-19 14:14:56,2020-04-23 14:14:20,3 days 23:59:24,122.637363,126.593407,40.5,43.5,"122045,126045,121048,129048,123051,127051,0000..."
3,128.571429,42.008289,129048,True,2020-04-21 18:15:19,2020-04-25 18:14:56,3 days 23:59:37,126.593407,130.549451,40.5,43.5,"126045,130045,125048,133048,127051,131051,0000..."
4,132.527473,42.008289,133048,True,2020-04-23 22:15:47,2020-04-28 02:15:10,4 days 03:59:23,130.549451,134.505495,40.5,43.5,"130045,134045,129048,136048,131051,135051,0000..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2442,220.000000,-87.045181,220177,True,2020-03-19 20:42:36,2020-04-03 04:43:25,14 days 08:00:49,200.000000,240.000000,-88.5,-85.5,"001180,191174,214174,236174,259174,180177,2601..."
2443,260.000000,-87.045181,260177,True,2020-03-25 16:41:00,2020-04-05 12:43:45,10 days 20:02:45,240.000000,280.000000,-88.5,-85.5,"001180,236174,259174,281174,304174,220177,3001..."
2444,300.000000,-87.045181,300177,True,2020-03-28 16:42:43,2020-04-09 04:42:26,11 days 11:59:43,280.000000,320.000000,-88.5,-85.5,"001180,259174,281174,304174,326174,260177,3401..."
2445,340.000000,-87.045181,340177,True,2020-03-31 20:42:37,2020-04-12 00:44:26,11 days 04:01:49,320.000000,360.000000,-88.5,-85.5,"001180,304174,326174,349174,011174,300177,0201..."


In [27]:
ecpv.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,Field_Name,Field_Type,active_insts
0,129.550000,1.500000,300007,True,2019-11-03 02:42:50,2019-11-04 03:36:37,89627.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
1,133.860000,1.500000,300008,True,2019-11-04 03:49:16,2019-11-05 05:16:39,91643.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
2,138.140000,1.500000,300009,True,2019-11-05 05:29:18,2019-11-06 06:40:06,90648.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
3,142.450000,1.500000,300010,True,2019-11-06 07:24:46,2019-11-07 08:20:08,89722.0,EFEDS,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
4,130.331300,-78.963400,300004,True,2019-11-16 23:14:40,2019-11-18 18:17:12,154952.0,ETA_CHA,SURVEY,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
...,...,...,...,...,...,...,...,...,...,...
165,284.146250,-37.909167,700008,True,2019-10-24 11:11:19,2019-10-25 08:55:52,78273.0,1RXS_J185635_375433,EXTRAGALACTIC_FIELDS,"TM1,TM2,TM3,TM4,TM5,TM6,TM7"
166,281.540771,79.873726,900060,True,2019-09-24 15:27:06,2019-09-24 21:30:32,21806.0,3C390_3,EXTRAGALACTIC_FIELDS,TM6
167,281.500275,79.885376,900068,True,2019-09-28 15:49:51,2019-09-28 21:30:32,20441.0,3C390_3,EXTRAGALACTIC_FIELDS,TM6
168,281.489410,79.888214,900069,True,2019-09-29 15:23:24,2019-09-29 21:30:37,22033.0,3C390_3,EXTRAGALACTIC_FIELDS,TM6


In [28]:
nu.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,proprietary_end_date,target_category,exposure_a,exposure_b,ontime_a,ontime_b,nupsdout,issue_flag,target_name
0,83.8281,-69.2466,40001014013,True,True,2013-06-29 01:16:10.560,2013-07-05 08:51:04.320,6 days 07:34:53.760000,2015-09-17,MISC,5 days 11:20:42,5 days 11:09:58,5 days 20:32:48,5 days 20:33:46,2119,1,SN1987A
1,83.7759,-69.2678,40001014016,True,True,2014-04-22 21:06:02.880,2014-04-29 09:31:06.240,6 days 12:25:03.360000,2015-09-17,MISC,4 days 23:56:47,4 days 23:35:17,5 days 08:14:46,5 days 08:14:09,0,1,SN1987A
2,83.8965,-69.2478,40001014023,True,True,2014-08-01 23:46:10.560,2014-08-08 02:41:08.160,6 days 02:54:57.600000,2015-09-17,MISC,4 days 22:46:56,4 days 22:35:32,5 days 07:01:07,5 days 07:02:55,0,0,SN1987A
3,340.6530,29.6985,60401031004,True,True,2018-11-28 22:21:12.960,2018-12-08 17:16:13.440,9 days 18:55:00.480000,2019-12-25,AGN,4 days 17:35:58,4 days 17:01:43,5 days 02:48:25,5 days 02:49:43,0,0,Ark_564
4,344.4048,-36.9766,60901012002,True,True,2023-11-02 06:06:11.520,2023-11-10 11:41:08.160,8 days 05:34:56.640000,2024-05-20,AGN,4 days 11:03:16,4 days 10:06:55,4 days 19:02:15,4 days 19:02:36,0,0,MS_22549m3712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4779,83.6747,22.0910,10013011001,True,True,2012-07-25 09:01:09.120,2012-07-25 10:36:02.880,0 days 01:34:53.760000,2014-09-23,CAL,0 days 00:00:44,0 days 00:00:47,0 days 00:01:40,0 days 00:01:40,0,1,Crab_oa11
4780,277.0668,-36.9712,90801012002,True,True,2022-06-09 18:11:05.280,2022-06-09 18:31:06.240,0 days 00:20:00.960000,2022-06-20,TOO,0 days 00:00:40,0 days 00:00:40,0 days 00:00:43,0 days 00:00:43,0,0,1A_1744m361_SADA_22159
4781,84.6870,-64.1111,10601308003,True,True,2020-04-07 21:14:15.360,2020-04-07 21:15:50.400,0 days 00:01:35.040000,2020-04-17,CAL,0 days 00:00:29,0 days 00:00:28,0 days 00:00:31,0 days 00:00:31,0,1,LMC_X3
4782,201.3151,-43.0243,00501005002,True,False,2019-09-06 00:41:11.040,2019-09-06 17:41:08.160,0 days 16:59:57.120000,NaT,MISC,0 days 00:00:00,0 days 08:44:21,0 days 00:00:00,0 days 09:35:09,0,1,Cen_A_FPMB


In [29]:
xr.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,target_category,exposure,xtend_exposure,resolve_mode,xtend_mode1-2,xtend_mode3-4,xtend_dataclass1-2,xtend_dataclass3-4
0,174.75746,-37.73904,300050010,True,False,2024-07-18 04:58:03.999996,2024-07-27 16:16:04.000002,9 days 11:18:00,MISC,5 days 05:20:10.395890,4 days 07:18:31.905700,PX_NORMAL,WINDOW2,WINDOW1,31100010,320000A0
1,194.94363,27.94653,300073010,True,False,2024-07-09 07:17:03.999998,2024-07-18 04:58:03.999996,8 days 21:41:00,MISC,4 days 15:41:41.078650,4 days 03:07:12.354140,PX_NORMAL,WINDOW1,,300000A0,
2,227.79431,5.82455,300053010,True,False,2024-07-27 16:16:04.000002,2024-08-04 19:17:03.999998,8 days 03:01:00,MISC,4 days 10:26:25.581890,4 days 01:17:20.064500,PX_NORMAL,WINDOW1,,300000A0,
3,83.86674,-69.26864,300021010,True,False,2024-06-17 18:20:04.000001,2024-06-22 14:01:04.000002,4 days 19:41:00,MISC,4 days 00:54:26.155770,2 days 00:58:03.841690,PX_NORMAL,WINDOW1,,300000A0,
4,287.79356,9.09746,300055010,True,False,2024-04-23 01:42:03.999998,2024-04-30 22:58:03.999996,7 days 21:16:00,MISC,3 days 20:22:06.908330,3 days 03:44:15.456360,PX_NORMAL,WINDOW1,,300000A0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,326.24898,38.36689,100009090,True,False,2024-04-21 22:17:03.999998,2024-04-22 01:32:04.000001,0 days 03:15:00,MISC,0 days 02:10:11.937500,0 days 00:01:37.860930,PX_NORMAL,WINDOW1BURST,,31200010,
212,326.18557,38.39637,100009130,True,False,2024-04-22 17:27:03.999998,2024-04-22 20:39:04.000003,0 days 03:12:00,MISC,0 days 02:08:15.937500,0 days 00:01:44.405190,PX_NORMAL,WINDOW1BURST,,31200010,
213,326.18113,38.37132,100009120,True,False,2024-04-22 14:15:04.000000,2024-04-22 17:27:03.999998,0 days 03:12:00,MISC,0 days 02:06:54,0 days 00:01:35.437130,PX_NORMAL,WINDOW1BURST,,31200010,
214,326.20292,38.33973,100009040,True,False,2024-04-20 23:54:03.999997,2024-04-21 03:18:04.000000,0 days 03:24:00,MISC,0 days 02:05:05,0 days 00:01:24.833010,PX_NORMAL,WINDOW1BURST,,31200010,


In [30]:
sw.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,xrt_exposure,bat_exposure,uvot_exposure,roll_angle
0,328.70739,16.89364,00173780007,True,2005-12-25 00:42:01.999999,2005-12-28 00:00:23.000000,2 days 23:18:21.000001,MISC,1 days 02:44:09.014000,1 days 00:04:10,1 days 02:40:01.030000,224.66184
1,39.95079,-25.21897,00131560002,True,2005-06-03 23:59:00.999998,2005-06-06 22:47:30.000002,2 days 22:48:29.000004,MISC,0 days 22:36:12.946000,0 days 22:52:03,0 days 17:46:44.297000,37.50837
2,228.46115,30.87659,00164268014,True,2005-12-03 01:00:00.999996,2005-12-06 00:07:43.000000,2 days 23:07:42.000004,MISC,0 days 19:38:55.384000,0 days 19:59:15,0 days 00:00:00,146.36382
3,350.64958,5.74990,00148833003,True,2005-08-06 00:26:11.999996,2005-08-09 00:50:56.000003,3 days 00:24:44.000007,MISC,0 days 19:04:07.125000,0 days 19:23:36,0 days 18:21:43.023000,49.28614
4,75.27823,45.28513,00321174011,True,2008-08-31 02:25:00.999998,2008-09-03 00:34:06.999998,2 days 22:09:06,MISC,0 days 18:49:28.970000,0 days 21:08:01.650000,0 days 18:24:15.039000,81.38752
...,...,...,...,...,...,...,...,...,...,...,...,...
373177,172.13889,-84.99158,00068196002,True,2005-10-02 11:44:21.999998,2005-10-02 11:48:22.000000,0 days 00:04:00.000002,MISC,0 days 00:00:00,0 days 00:00:13,0 days 00:00:00,16.33025
373178,57.33072,-85.00272,00067386002,True,2005-05-26 23:59:00.999998,2005-05-27 01:23:15.000000,0 days 01:24:14.000002,MISC,0 days 00:00:00,0 days 00:02:35,0 days 00:00:00,6.33332
373179,172.06705,-85.00961,00074488015,True,2007-04-03 05:10:59.999998,2007-04-03 06:47:40.000004,0 days 01:36:40.000006,MISC,0 days 00:00:00,0 days 00:02:51,0 days 00:00:00,200.09891
373180,57.06398,-85.02518,00074476005,True,2006-11-29 23:33:01.000002,2006-11-30 01:19:36.000002,0 days 01:46:35,MISC,0 days 00:00:00,0 days 00:02:26,0 days 00:00:00,188.38622


In [31]:
ra.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category
0,263.57088,67.500,RS930521N00,True,1990-07-11,1991-08-13,0 days 11:19:06,ASK
1,276.42533,67.500,RS930522N00,True,1990-07-11,1991-08-13,0 days 11:18:11,ASK
2,96.42533,-67.500,RS932908N00,True,1990-07-11,1991-08-13,0 days 05:47:48,ASK
3,83.57088,-67.500,RS932907N00,True,1990-07-11,1991-08-13,0 days 05:47:39,ASK
4,267.27100,61.875,RS930625N00,True,1990-07-11,1991-08-13,0 days 03:44:35,ASK
...,...,...,...,...,...,...,...,...
1373,289.08783,-61.875,RS932827N00,True,1990-09-10,1990-10-08,0 days 00:02:44,ASK
1374,278.17942,-61.875,RS932826N00,True,1990-09-04,1990-09-30,0 days 00:02:42,ASK
1375,273.75000,-45.000,RS932537N00,True,1990-09-07,1990-09-23,0 days 00:02:42,ASK
1376,350.17942,-33.750,RS932354N00,True,1990-11-09,1990-12-02,0 days 00:02:42,ASK


In [32]:
rp.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,instrument,with_filter,target_category,target_name,proc_rev,fits_type
0,163.18,57.48,RH701867A01,True,1995-04-15 23:24:16.000001,1995-05-11 14:24:47.000001,2 days 13:29:32,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V4.
1,203.65,37.91,RH900717N00,True,1997-06-04 16:13:00.999998,1997-07-13 22:26:45.000004,2 days 07:58:33,HRI,N,MISC,DEEP SURVEY,2,RDF 4_0
2,163.18,57.48,RH701867A04,True,1997-04-15 21:51:16.999998,1997-04-28 16:37:45.999998,2 days 06:33:41,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V3.
3,350.87,58.81,RH500444N00,True,1995-12-23 22:18:36.999999,1996-02-01 10:17:53.999998,2 days 02:07:27,HRI,N,SNR,CAS A,2,RDF 3_4
4,163.18,57.48,RH701867A02,True,1996-05-01 02:09:33.000002,1996-05-29 15:31:24.999997,2 days 01:05:24,HRI,N,AGN,LOCKMAN HOLE,2,RFITS V4.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,107.17,-49.55,RH703919N00,False,1998-12-02 20:07:49.000002,1998-12-02 21:52:51.000001,0 days 00:00:00,HRI,N,AGN,,2,RFITS V4.
11427,107.17,-49.55,RH703918N00,False,1998-12-01 21:47:20.999996,1998-12-01 23:37:05.999998,0 days 00:00:00,HRI,N,AGN,,2,RFITS V4.
11428,107.17,-49.55,RH703920N00,False,1998-12-03 19:53:49.000004,1998-12-03 20:07:53.999996,0 days 00:00:00,HRI,N,AGN,,2,RFITS V4.
11429,107.17,-49.55,RH703921N00,False,1998-12-04 19:49:02.000000,1998-12-04 21:35:54.000004,0 days 00:00:00,HRI,N,AGN,,2,RFITS V4.


In [33]:
su.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,xis0_expo,xis0_num_modes,xis1_expo,xis1_num_modes,xis2_expo,xis2_num_modes,xis3_expo,xis3_num_modes
0,91.1523,-86.6779,701018010,True,2006-04-13 16:24:07.999998,2006-04-14 01:52:19.000001,0 days 09:28:11.000003,EGS,0 days 05:29:41.700000,2,0 days 05:29:41.700000,2,0 days 05:29:25.700000,2,0 days 05:29:33.700000,2
1,48.9864,-85.5003,404019010,True,2009-07-16 14:26:31.000001,2009-07-18 03:30:15.999998,1 days 13:03:44.999997,GS,0 days 17:31:24.300000,2,0 days 17:31:24.300000,2,0 days 00:00:00,0,0 days 17:31:24.300000,2
2,239.2836,-79.2302,703059010,True,2008-10-13 15:35:23.999997,2008-10-15 21:37:21.000000,2 days 06:01:57.000003,EGS,0 days 22:49:57,2,0 days 22:49:43.500000,2,0 days 00:00:00,0,0 days 22:49:35.500000,2
3,265.9438,-76.3446,705013010,True,2010-04-14 00:16:05.000002,2010-04-14 16:45:11.000002,0 days 16:29:06,EGS,0 days 11:46:23.600000,2,0 days 11:46:18.300000,2,0 days 00:00:00,0,0 days 11:46:29,2
4,74.6207,-75.2810,404036010,True,2009-06-14 01:51:18.000003,2009-06-16 13:27:19.000002,2 days 11:36:00.999999,GS,1 days 05:42:37.800000,2,1 days 05:42:37.800000,2,0 days 00:00:00,0,1 days 05:42:29.800000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,247.6537,82.9256,706004010,True,2011-04-16 00:38:47.000003,2011-04-16 10:23:15.000000,0 days 09:44:27.999997,EGS,0 days 05:13:55.700000,3,0 days 05:13:42,2,0 days 00:00:00,0,0 days 05:13:55.600000,2
3037,116.7261,85.6829,804031010,True,2010-02-08 09:53:43.999999,2010-02-08 18:15:19.000002,0 days 08:21:35.000003,GCL,0 days 05:19:56,1,0 days 05:19:56,1,0 days 00:00:00,0,0 days 05:19:56,1
3038,112.8158,85.6965,804030010,True,2010-02-08 18:18:12.000001,2010-02-09 12:00:25.000004,0 days 17:42:13.000003,GCL,0 days 10:17:39.100000,2,0 days 10:17:39.100000,2,0 days 00:00:00,0,0 days 10:17:39.100000,2
3039,112.8340,85.6974,804030020,True,2010-02-10 04:18:27.999997,2010-02-11 06:00:24.000002,1 days 01:41:56.000005,GCL,0 days 14:36:01.100000,2,0 days 14:36:01.100000,2,0 days 00:00:00,0,0 days 14:36:01.100000,2


In [34]:
asca.all_obs_info

,ra,dec,ObsID,science_usable,start,end,duration,target_category,sis_exposure,gis_exposure
0,288.0116,5.0542,48002000,True,2000-03-23 05:52:13.440,2000-04-04 00:00:25.920,11 days 18:08:12.480000,XRB,5 days 07:25:04,5 days 14:47:28
1,253.5540,39.8158,78002000,True,2000-03-01 12:08:12.480,2000-03-11 00:00:51.840,9 days 11:52:39.360000,AGN,4 days 00:45:04,5 days 04:32:16
2,14.3115,-22.4431,77036000,True,1999-12-03 11:50:38.400,1999-12-15 19:00:28.800,12 days 07:09:50.400000,AGN,4 days 08:14:08,5 days 00:42:40
3,203.8856,-34.3378,77003000,True,1999-07-19 00:40:19.200,1999-07-29 15:40:27.840,10 days 15:00:08.640000,AGN,4 days 02:03:12,4 days 16:33:20
4,266.4864,-28.9429,48004000,True,2000-03-11 00:08:29.760,2000-03-22 00:00:43.200,10 days 23:52:13.440000,XRB,3 days 22:37:04,4 days 07:30:24
...,...,...,...,...,...,...,...,...,...,...
3069,87.7568,-32.2321,77010010,True,1999-10-11 13:47:08.160,1999-10-11 14:19:32.160,0 days 00:32:24,AGN,0 days 00:06:40,0 days 00:06:40
3070,150.6550,-58.6295,15000090,True,1993-08-05 18:31:58.080,1993-08-05 19:12:43.200,0 days 00:40:45.120000,GS,0 days 00:06:56,0 days 00:05:52
3071,333.8738,-17.8223,77002000,True,1999-11-17 04:45:41.760,1999-11-19 01:35:54.240,1 days 20:50:12.480000,AGN,0 days 16:30:24,0 days 00:00:00
3072,335.1555,-24.7349,66013000,True,1998-11-16 16:24:48.960,1998-11-18 17:34:39.360,2 days 01:09:50.400000,NGS,0 days 14:38:24,0 days 00:00:00


In [35]:
inte.all_obs_info

,ra,dec,ObsID,science_usable,proprietary_usable,start,end,duration,target_category,jemx1_exposure,jemx2_exposure,isgri_exposure,picsit_exposure,spi_exposure,scw_ver
0,294.97232,-89.65222,218500510010,True,True,2020-01-22 20:40:39.751343,2020-01-22 22:17:46.754676,0 days 01:37:07.003333,MISC,0 days 01:36:12,0 days 01:36:12,0 days 01:36:10,0 days 00:28:29,0 days 01:36:12,001
1,117.47166,-89.57144,218600510010,True,True,2020-01-25 11:40:02.881811,2020-01-25 12:38:50.883829,0 days 00:58:48.002018,MISC,0 days 00:57:57,0 days 00:57:57,0 days 00:57:55,0 days 00:57:27,0 days 00:57:57,001
2,166.19875,-89.54264,229200810010,True,True,2020-11-02 08:42:24.928042,2020-11-02 09:41:14.930022,0 days 00:58:50.001980,MISC,0 days 00:58:00,0 days 00:58:00,0 days 00:57:44,0 days 00:57:29,0 days 00:58:00,001
3,130.24533,-89.51414,209300110010,True,True,2019-05-21 07:51:13.039554,2019-05-21 08:24:33.041123,0 days 00:33:20.001569,MISC,0 days 00:32:25,0 days 00:32:25,0 days 00:32:23,0 days 00:32:04,0 days 00:32:25,001
4,125.61337,-89.47669,209300370010,True,True,2019-05-21 22:51:03.082145,2019-05-21 23:24:24.083715,0 days 00:33:21.001570,MISC,0 days 00:32:30,0 days 00:32:30,0 days 00:32:28,0 days 00:32:02,0 days 00:32:30,001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205738,268.11816,89.32161,260700500010,True,True,2023-02-17 10:30:57.695328,2023-02-17 11:00:58.696476,0 days 00:30:01.001148,MISC,0 days 00:29:10,0 days 00:29:10,0 days 00:29:10,0 days 00:28:47,0 days 00:29:12,001
205739,177.31691,89.40664,267800540010,True,True,2023-08-25 08:38:46.274689,2023-08-25 09:08:48.275969,0 days 00:30:02.001280,MISC,0 days 00:28:50,0 days 00:28:50,0 days 00:29:08,0 days 00:00:00,0 days 00:29:10,001
205740,183.68471,89.46394,267600300010,True,True,2023-08-19 11:23:45.855551,2023-08-19 11:54:48.857366,0 days 00:31:03.001815,MISC,0 days 00:30:08,0 days 00:30:08,0 days 00:30:06,0 days 00:29:47,0 days 00:30:08,001
205741,185.45596,89.47636,267600290010,True,True,2023-08-19 11:11:57.854864,2023-08-19 11:21:50.855440,0 days 00:09:53.000576,MISC,0 days 00:09:34,0 days 00:09:34,0 days 00:09:32,0 days 00:00:00,0 days 00:09:34,001


### Filter Array

This is unlikely to ever be accessed directly by the user, but is what defines the observations that a mission currently deems to be accepted/selected. It is a boolean numpy array with a length equal to the number of observations in the all_obs_info dataframe, a `True` value means the observation is accepted and a `False` value means it is excluded; all observations start off as accepted.

Various filtering methods can be used to adjust the filter array and set the observations which are to be downloaded/included in a DAXA archive, depending on your particular sample and science case. 

It is also possible to manually set this filter array, as is demonstrated below:

In [36]:
# The filter array defaults to all True, so all observations are accepted
xm.filter_array

array([ True,  True,  True, ...,  True,  True,  True])

In [37]:
# Demonstrating manually setting a filter array - it must be boolean and be the same length as the
#  'all_obs_info' table, otherewise it will not be accepted
demo_filt_arr = np.full(len(xm.all_obs_info), True)
demo_filt_arr[0] = False
xm.filter_array = demo_filt_arr
xm.filter_array

array([False,  True,  True, ...,  True,  True,  True])

## Selecting relevant observations

Few users will wish to download, process, and maintain ___complete___ observation archives, preferring to just locate data that may be relevant to the sources that they are studying. This can be achieved with the use of several filtering methods which are built into all DAXA missions.

Here we introduce the different filtering methods that are currently implemented for DAXA missions, but we do not provide detailed demonstrations of their use; that is [left to specific case studies](../../tutorials.casestudies.html) designed to show scientists with different needs how DAXA can be used in ways that are most relevant to them.

### Filtering on ObsID

The most basic filtering method available can be used when you already know which observation(s) you are interested in - if you have the ObsID(s) you can just pass them to the `filter_on_obs_ids` method, and select only that data.

**Alternatively** you can invert that behaviour, and select every observation that _isn't_ one of the specified ObsIDs, by passing `invert=True`.

In [3]:
help(xm.filter_on_obs_ids)

Help on method filter_on_obs_ids in module daxa.mission.base:

filter_on_obs_ids(allowed_obs_ids: Union[str, List[str]], invert: bool = False) method of daxa.mission.xmm.XMMPointed instance
    This filtering method will select only observations with IDs specified by the allowed_obs_ids argument (or
    select only observations that AREN'T specified by the allowed_obs_ids argument, if invert=True).
    
    Please be aware that filtering methods are cumulative, so running another method will not remove the
    filtering that has already been applied, you can use the reset_filter method for that.
    
    :param str/List[str] allowed_obs_ids: The ObsID (or list of ObsIDs) that you wish to be let
        through the filter (or exclude from it if invert=True).
    :param bool invert: This argument can invert the behaviour of this filtering method, selecting every
        ObsID that isn't specified in allowed_obs_ids. Default is False.



### Filtering on position

Arguably the most useful type of filtering supported by DAXA missions, the `filter_on_positions` method allows us to search for observations that are relevant to specific positions on the sky (generally these will represent particular objects). A search can be performed either for a single position, or for a whole sample.

The `search_distance` argument controls how close the central coordinate of an observation must be to a search position for that observation to be accepted. The default search positions are defined by the field-of-view of the telescope (if a mission has multiple instruments with different field-of-views then each instrument will be searched with the correct field-of-view). The user may also specify their own search distance value (or values), and it is also possible to specify a different search distance for every position.

The `return_pos_obs_info` argument controls whether a dataframe is returned that links passed positions to particular observations that are relevant to them - this dataframe would not include positions that are determined to have no relevant observations.

In [39]:
help(xm.filter_on_positions)

Help on method filter_on_positions in module daxa.mission.base:

filter_on_positions(positions: Union[list, numpy.ndarray, astropy.coordinates.sky_coordinate.SkyCoord], search_distance: Union[astropy.units.quantity.Quantity, float, int, list, numpy.ndarray, dict] = None, return_pos_obs_info: bool = False) -> Optional[pandas.core.frame.DataFrame] method of daxa.mission.xmm.XMMPointed instance
    This method allows you to filter the observations available for a mission based on a set of coordinates for
    which you wish to locate observations. The method searches for observations by the current mission that have
    central coordinates within the distance set by the search_distance argument.

    Please be aware that filtering methods are cumulative, so running another method will not remove the
    filtering that has already been applied, you can use the reset_filter method for that.

    :param list/np.ndarray/SkyCoord positions: The positions for which you wish to search for observa

### Filtering on name

If you are interested in data relevant to named objects, you can pass the name(s) to the `filter_on_name` method. It will use a lookup service (specifically Sesame), to locate coordinates for the object(s), and then pass that to the `filter_on_positions` method. 

Bear in mind that you are reliant on the lookup service having accurate central coordinates for the named object, so it could be worth checking with your own coordinates and using `filter_on_positions` directly if you can't find any observations!

In [40]:
help(xm.filter_on_name)

Help on method filter_on_name in module daxa.mission.base:

filter_on_name(object_name: Union[str, List[str]], search_distance: Union[astropy.units.quantity.Quantity, float, int, list, numpy.ndarray, dict] = None, parse_name: bool = False) method of daxa.mission.xmm.XMMPointed instance
    This method wraps the 'filter_on_positions' method, and allows you to filter the mission's observations so
    that it contains data on a single (or a list of) specific objects. The names are passed by the user, and
    then parsed into coordinates using the Sesame resolver. Those coordinates and the search distance are
    then used to find observations that might be relevant.

    :param str/List[str] object_name: The name(s) of objects you would like to search for.
    :param Quantity/float/int/list/np.ndarray/dict search_distance: The distance within which to search for
        observations by this mission. Distance may be specified either as an Astropy Quantity that can be
        converted to d

### Filtering on target type

It is possible to filter observations based on the type of object that was the original target of the observation. **Warning: this should be used with significant caution, as our object taxonomy may not be granular enough to represent all different types of astronomical objects, and conversions between the different target types used by different missions and our target types is imperfect!**

Here we display the DAXA source type taxonomy, the short form codes on the left are what should be passed to the `filter_on_target_type` method - the user may pass either a single target type, or a list of them:

In [41]:
xm.show_allowed_target_types()

╒═══════════════╤═══════════════════════════════════════════════╕
│ Target Type   │ Description                                   │
╞═══════════════╪═══════════════════════════════════════════════╡
│ AGN           │ Active Galaxies and Quasars                   │
├───────────────┼───────────────────────────────────────────────┤
│ BLZ           │ Blazars                                       │
├───────────────┼───────────────────────────────────────────────┤
│ CV            │ Cataclysmic Variables                         │
├───────────────┼───────────────────────────────────────────────┤
│ CAL           │ Calibration Observation (possibly of objects) │
├───────────────┼───────────────────────────────────────────────┤
│ EGS           │ Extragalactic Surveys                         │
├───────────────┼───────────────────────────────────────────────┤
│ GCL           │ Galaxy Clusters                               │
├───────────────┼───────────────────────────────────────────────┤
│ GS      

In [42]:
help(xm.filter_on_target_type)

Help on method filter_on_target_type in module daxa.mission.base:

filter_on_target_type(target_type: Union[str, List[str]]) method of daxa.mission.xmm.XMMPointed instance
    This method allows the filtering of observations based on what type of object their target source was. It
    is only supported for missions that have that data available, and will raise an exception for those
    missions that don't support this filtering.

    some may be incorrect. They also don't take into account sources that might serendipitously appear in
    a particular observation.

    :param str/List[str] target_type: The types of target source you would like to find observations of. For
        allowed types, please use the 'show_allowed_target_types' method. Can either be a single type, or
        a list of types.



### Filtering on time

Observations can be filtered on ___when___ they were taken, with the user specifying a time frame (defined by a start and end date-time) from which they wish to select observations. By default any observation that coincides with that time window (either starting in it, ending in it, or starting and ending outside but being taken during it) will be selected - it is also possible to require that an observation must have taken place entirely within the time window.

___Warning: Observations of survey missions like eRASS1DE and ROSATAllSky may repeatedly visit a particular location, and all that data may be incorporated as one observation, meaning that there may not constant coverage in such an observation___

In [43]:
help(xm.filter_on_time)

Help on method filter_on_time in module daxa.mission.base:

filter_on_time(start_datetime: datetime.datetime, end_datetime: datetime.datetime, over_run: bool = True) method of daxa.mission.xmm.XMMPointed instance
    This method allows you to filter observations for this mission based on when they were taken. A start
    and end time are passed by the user, and observations that fall within that window are allowed through
    the filter. The exact behaviour of this filtering method is controlled by the over_run argument, if set
    to True then observations with a start or end within the search window will be selected, but if False
    then only observations with a start AND end within the window are selected.

    Please be aware that filtering methods are cumulative, so running another method will not remove the
    filtering that has already been applied, you can use the reset_filter method for that.

    :param datetime start_datetime: The beginning of the time window in which to s

### Filtering on time & position

A method of filtering that combines positional and temporal filtering, so that observations of a particular position, within a particular time window, can be located. The user ___does not___ have to filter for one position-time combination at a time - they may pass a set of positions, with a corresponding set of time windows, and find all the observations that fulfill those requirements.

The `filter_on_positions_at_time` method supports the same arguments passed to `filter_on_positions` and `filter_on_time`, which set search distances from the passed coordinate (`search_distance`), whether a dataframe linking particular input coordinates to particular selected observations (`return_obs_info`), and whether only observations that start and end within the specified time period should be considered (`over_run`). 

In [44]:
help(xm.filter_on_positions_at_time)

Help on method filter_on_positions_at_time in module daxa.mission.base:

filter_on_positions_at_time(positions: Union[list, numpy.ndarray, astropy.coordinates.sky_coordinate.SkyCoord], start_datetimes: Union[numpy.ndarray, datetime.datetime], end_datetimes: Union[numpy.ndarray, datetime.datetime], search_distance: Union[astropy.units.quantity.Quantity, float, int, list, numpy.ndarray, dict] = None, return_obs_info: bool = False, over_run: bool = True) method of daxa.mission.xmm.XMMPointed instance
    This method allows you to filter the observations available for a mission based on a set of coordinates for
    which you wish to locate observations that were taken within a certain time frame. The method spatially
    searches for observations that have central coordinates within the distance set by the search_distance
    argument, and temporally by start and end times passed by the user; and observations that fall within that
    window are allowed through the filter.

    The exact b

## Downloading data

Once the user has decided upon a set of observations, for a particular mission, that are relevant to their research - the next step is often to download them. This section specifies what can be downloaded, and how to download it.

### What can be downloaded?

The exact data that can be downloaded depends on what a particular mission has made available on their online archive - Event lists are available for all missions bar INTEGRAL, and most missions support the acquisition of pre-generated images, but not all (INTEGRAL and eROSITA CalPV) for instance).

We make a distinction between 'raw' and 'pre-processed' data, which is necessarily fuzzy due to the disparate natures of the various data archives we have to deal with. Broadly speaking '**raw**' data means either absolutely raw files that need to be processed into initial event lists (the case with XMMPointed) or just event lists (uncleaned and pre-cleaned); '**pre-processed**' data includes pre-processed data products such as images, exposure maps, and background maps.

**This means that the user can either set up an archive using the pre-processed data, or re-process data using DAXA interfaces to the various telescope backend software packages.**

Pre-processed products are downloaded by default (in addition to the raw data), but when setting up an Archive you can choose between using pre-processed data or re-processing the raw data; if you do not wish to acquire pre-processed data products, you can pass `download_products=False` to the download method (see below). 

**Event lists are considered raw data (i.e. not pre-processed) for most missions, and will always be downloaded**

#### XMM-Newton

_Currently_ we only support the acquisition of raw data.

#### Chandra

The following data products can be downloaded:

* Full FoV image (if an Archive is constructed including pre-processed Chandra data, this is what is included).
* High-resolution central image.

#### eROSITA All-Sky DR1 (German Half)

The following data products can be downloaded:

* Images
* Exposure maps
* Background maps

#### eROSITA Calibration and Performance Verification

No pre-processed data products are available, only event lists.

#### NuSTAR-Pointed

The following data products can be downloaded:

* Images

#### Swift

The following data products can be downloaded (_no products are available for BAT, only raw data_):

* Images (XRT and UVOT)
* Exposure maps (XRT and UVOT)

#### ROSAT All-Sky Survey

The following data products can be downloaded:

* Images
* Exposure maps

#### ROSAT-Pointed

The following data products can be downloaded:

* Images
* Exposure maps (only PSPC, no HRI)

#### Suzaku

The following data products can be downloaded:

* Images

#### ASCA

The following data products can be downloaded:

* Images (SIS0 + SIS1, and GIS2 + GIS3)
* Exposure maps
* Lightcurves (not included in Archive constructed from pre-processed data)
* Spectra (not included in Archive constructed from pre-processed data)

#### INTEGRAL-Pointed

No pre-processed data (nor even event lists) are available to download for INTEGRAL, only raw data and calibration files.

### How can it be downloaded?

The selected data can be downloaded using the `download` method that is built into each mission class.

If multiple observations have been selected, then downloads can be multi-threaded; this can sometimes offer a speed benefit, though only if the archive and user internet connections are not saturated by the download. The number of cores used can be set via the `num_cores` argument; by defualt this is set to the NUM_CORES DAXA constant, which can either be set by the user in the DAXA configuration file/by setting the value of daxa.NUM_CORES, or will be 90% of the cores of the system.

**Note that it is not necessary to manually activate the download method if you will be creating a DAXA Archive from the filtered mission objects, as that will be done on archive initialisation - [see the Archive tutorial for more information.](archives.html)**

Here we demonstrate a simple XMM download, as well as a Chandra download that includes pre-generated images (standard Chandra reprocessing scripts can be used on this downloaded data):

In [45]:
xm.filter_on_obs_ids('0201903501')
xm.download(num_cores=1)

ch.filter_on_obs_ids('3205')
ch.download(num_cores=1, download_products=True)

## Getting paths to downloaded mission data products

If pre-processed data have been downloaded, it is possible to use methods built into the mission class to retrieve the paths to various data products. **If you intend on using an XGA archive, using these methods should not be necessary** as the data will be moved to the Archive processed data storage structure, but it may still be useful if you just wish to use DAXA to download data.

There are four different get methods:

* `get_evt_list_path` - to retrieve event lists.
* `get_image_path` - to retrieve images.
* `get_expmap_path` - to retrieve exposure maps.
* `get_background_path` - to retrieve background maps.

They are very easy to use - only the ObsID, instrument (in most cases, not necessary if the mission has only one instrument per ObsID), and lower/upper energy bound (for images, exposure maps, and background maps - though if the products only have one energy band it will be completed automatically) need to be provided. They also provide helpful, detailed, error messages if what is requested isn't possible.

Here we retrive the path to the event list (note that we do not need to pass the instrument name, as Chandra has only one per ObsID):

In [46]:
ch.get_evt_list_path('3205')

'/Users/dt237/code/DAXA/docs/source/notebooks/tutorials/daxa_output/chandra_raw/3205/primary/acisf03205N006_evt2.fits'

We can also retrieve the path to the full-FoV image (again we don't need to pass instrument in this specific instance, nor do we need to pass energy bounds because Chandra only supplies one): 

In [47]:
ch.get_image_path('3205')

'/Users/dt237/code/DAXA/docs/source/notebooks/tutorials/daxa_output/chandra_raw/3205/primary/acisf03205N006_full_img2.fits'

A more typical way of using this function would be this:

In [48]:
ch.get_image_path('3205', Quantity(0.5, 'keV'), Quantity(7, 'keV'), 'ACIS-I')

/Users/dt237/code/DAXA/daxa/mission/base.py:942: UserWarning: Some instrument names were converted to alternative forms expected by this module, the instrument names are now; ACIS
  inst = self.check_inst_names(inst)[0]


'/Users/dt237/code/DAXA/docs/source/notebooks/tutorials/daxa_output/chandra_raw/3205/primary/acisf03205N006_full_img2.fits'

The get methods for exposure and background map paths operate identically to the image path method, though will show an error for Chandra:

In [49]:
ch.get_expmap_path('3205', Quantity(0.5, 'keV'), Quantity(7, 'keV'), 'ACIS-I')

PreProcessedNotSupportedError: This mission (Chandra) does not support the download of pre-processed exposure maps, so a path cannot be provided. 

## Saving mission state

It is possible to save the current 'state' of a mission object to a file - which in turn can be used to reload a mission class and have it be in that same state. This is primarily used by internal DAXA methods, to allow saved archives to reinstate their constituent mission classes when they are loaded back in.

However, it is also possible to do this manually, in order to reload missions as they were when they were saved. The save file could also be shared with other users, to allow them to create a mission in the same state.

**The state of the mission includes the filtering procedures performed upon it, and the current filtered data** - this means that the observations selected when the mission was saved will be reloaded exactly, and the same filtering steps can be re-applied on reload to update the mission (if more observations have become available).

Saving a mission state is simple - call the `save()` method and pass a root storage directory and optionally a file name (if this is not supplied the save file will be the mission name + '\_state.json'):

In [50]:
xm.save("demo_save/", "demo_xmm_save.json")

A mission can then be reinstated from the file on declaration:

In [51]:
reinstated_xm = XMMPointed(save_file_path="demo_save/demo_xmm_save.json")
reinstated_xm.filtered_obs_info

/Users/dt237/code/DAXA/daxa/mission/xmm.py:83: UserWarning: 140 of the 18126 observations located for this mission have been removed due to NaN RA or Dec values
  self._fetch_obs_info()


,ra,dec,ObsID,start,science_usable,duration,proprietary_end_date,revolution,proprietary_usable,end
3527,149.592285,-11.0597,0201903501,2004-06-17 10:38:28,True,0 days 04:08:33,2004-06-17,828,True,2004-06-17 14:47:01


If some time has passed since the mission state was saved, and you wish to re-run the original filtering procedures to check if new data are available, you can call the `update()` method.